# Société 

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import random

Dans la première matrice on place les consommateurs de drogues (modélisé avec le chiffre 1) et les non-consommateurs (modélisé avec le chiffre 0). 
On a d'abord une matrice composée uniquement de non-consommateurs, puis la répartition des consommateurs se fait grace a une certaine probabilité qu'on applique 

In [31]:
def tableauBase(P):
    """les 0 sont les non-cosommateurs et 1 les consommateurs."""
    tableau_de_zero_un = np.zeros((20,20), dtype='i') #np.zeros : renvoie un tableau 1D de 20 zero (lignes) et 20 zero (colonnes)
    for i in range(len(tableau_de_zero_un)): 
        for j in range(len(tableau_de_zero_un)):
            if random.random() < P :
                tableau_de_zero_un[i,j] = 1

    print(tableau_de_zero_un)
    
    return tableau_de_zero_un
    
tab_base = tableauBase(0.6)



[[1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1]
 [0 0 0 0 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0 0]
 [1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0]
 [1 1 1 0 1 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0]
 [0 1 0 0 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1]
 [1 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 1 1 0]
 [1 1 1 1 1 1 0 0 1 0 1 1 0 0 0 1 1 0 1 0]
 [0 0 1 0 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1]
 [0 1 1 0 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 0]
 [0 1 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 1]
 [0 1 0 1 1 1 1 0 1 1 0 0 1 0 1 0 0 0 0 1]
 [0 0 0 1 0 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1]
 [0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1]
 [1 0 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0]
 [1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1]
 [0 1 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1]
 [1 0 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0]
 [0 1 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 0 0 0]]


In [35]:
def verifNb(tabF):
    tab = tabF
    comptC = 0 #compteur consommateurs 
    comptNC = 0 #compteur non-consommateurs 
    for i in range(len(tab)) : #parcours de la matrice 
        for j in range(len(tab)) :#parcours de la matrice 
            if  i+1 <= len(tab) and i-1 >= 0 and  j+1 <= len(tab) and j-1 >= 0:
                if tab[i,j] == 1:
                    comptC = comptC +1
                if tab[i,j] == 0:
                    comptNC = comptNC +1
    return comptC + comptNC


print("nb pers :",verifNb(tab_base))

nb pers : 361
